In [1]:
import pandas as pd
from pathlib import Path
from read_data import clean_text

# load datasets

## new dataset

In [2]:
path="AI-GA/ai-ga-dataset.csv"

In [3]:
df_new=pd.read_csv(path)

### info:
- label 0 :: original abstract
- label 1 :: generated abstract

## work dataset

In [4]:
!wget https://github.com/vijini/GeneratedTextDetection/archive/refs/heads/main.zip
!unzip main

--2025-02-27 14:39:25--  https://github.com/vijini/GeneratedTextDetection/archive/refs/heads/main.zip
Resolving github.com (github.com)... 20.26.156.215
Connecting to github.com (github.com)|20.26.156.215|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/vijini/GeneratedTextDetection/zip/refs/heads/main [following]
--2025-02-27 14:39:25--  https://codeload.github.com/vijini/GeneratedTextDetection/zip/refs/heads/main
Resolving codeload.github.com (codeload.github.com)... 20.26.156.216
Connecting to codeload.github.com (codeload.github.com)|20.26.156.216|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘main.zip’

main.zip                [ <=>                ] 800.25K  --.-KB/s    in 0.07s   

2025-02-27 14:39:25 (10.6 MB/s) - ‘main.zip’ saved [819461]

Archive:  main.zip
ab034465f857a93212a894fe598edb749345b6ff
   creating: GeneratedTextDetection-main/
  inflating: 

In [4]:

root = Path("GeneratedTextDetection-main/Dataset/Hybrid_AbstractDataset")

train_texts, train_labels, test_texts, test_labels = [], [], [], []
for path in root.glob("*.txt"):
    with open(path, 'rt') as file:
        text = file.read()
        text = text.lstrip('\ufeff')
    label = int(path.name.endswith("generatedAbstract.txt"))
    doc_id = int(path.name.split("_")[0].split(".")[-1])
    if doc_id < 10522:
        test_texts.append(text)
        test_labels.append(label)
    else:
        train_texts.append(text)
        train_labels.append(label)

In [16]:
len(train_texts), len(train_labels)

(160, 160)

### info work dataset
- label 0 :: original abstract
- label 1 :: generated abstract

In [10]:
df_work_train = pd.DataFrame({
    "text": train_texts,
    "label": train_labels
})

In [11]:
df_work_test = pd.DataFrame({
    "text": test_texts,
    "label": test_labels
})

# Information

## on new data

In [6]:
df_new["label"].value_counts()

label
0    14331
1    14331
Name: count, dtype: int64

## work data

In [17]:
df_work_train["label"].value_counts()

label
1    80
0    80
Name: count, dtype: int64

In [18]:
df_work_test["label"].value_counts()

label
1    20
0    20
Name: count, dtype: int64

# clean new data set

In [8]:
df_new=df_new.drop(columns=['title'])

In [9]:
df_new["abstract"]=df_new["abstract"].apply(lambda abst: clean_text(abst))

In [19]:
df_new.columns

Index(['abstract', 'label'], dtype='object')

In [20]:
df2 = df_new.rename(columns={'abstract': 'text'})

In [22]:
df2.columns

Index(['text', 'label'], dtype='object')

# create the new dataset

## training

In [27]:
# selection aléatoire des valoir constituant le data set training
df2_label_0_tr = df2[df2['label'] == 0].head(n=4920 )
df2_label_1_tr = df2[df2['label'] == 1].head(n=4920)
df2_sampled_tr = pd.concat([df2_label_0_tr, df2_label_1_tr])

#combined the new and the work dataset
df_combined_tr = pd.concat([df2_sampled_tr, df_work_train])
df_final_tr = df_combined_tr.sample(frac=1, random_state=1).reset_index(drop=True)
print(df_final_tr["label"].value_counts())
df_final_tr.count()


label
1    5000
0    5000
Name: count, dtype: int64


text     10000
label    10000
dtype: int64

In [29]:
df_final_tr["status"]='train'

## test

In [28]:
# selection aléatoire des valoir constituant le data set training
df2_label_0_ts = df2[df2['label'] == 0].tail(n=1230 )
df2_label_1_ts = df2[df2['label'] == 1].tail(n=1230)
df2_sampled_ts = pd.concat([df2_label_0_ts, df2_label_1_ts])

#combined the new and the work dataset
df_combined_ts = pd.concat([df2_sampled_ts, df_work_test])
df_final_ts = df_combined_ts.sample(frac=1, random_state=1).reset_index(drop=True)
print(df_final_ts["label"].value_counts())
df_final_ts.count()

label
1    1250
0    1250
Name: count, dtype: int64


text     2500
label    2500
dtype: int64

In [31]:
df_final_ts["status"]='test'

# final data set

In [34]:
df_combined = pd.concat([df_final_tr, df_final_ts])

# Sauvegarder le DataFrame combiné en CSV
df_combined.to_csv('combined_dataset.csv', index=False)